In [ ]:
# llama mistrols

import requests

def llama3_prompt(api_key, prompt_text):
    url = "https://api.llama3.example.com/v1/prompt"  # Replace with the actual API endpoint
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "prompt": prompt_text,
        "max_tokens": 100  # Adjust as needed
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json()  # Return the JSON response
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
prompt_text = "What is the capital of France?"
response = llama3_prompt(API_KEY, prompt_text)

if response:
    print("Response from Llama3:", response)